In [ ]:
import pandas as pd
import numpy as np

data_root = '../../MyData/'

## Data pre-processing

### Load data

In [2]:
# Loading data
# Obtain compustat data from 2021
file_path = data_root + 'compustat_2021.csv'
compustat2021 = pd.read_csv(file_path)
compustat2021 = compustat2021.rename(columns={'fyear': 'Year'})
print(len(compustat2021))

# remove rows that contain nan in 'cik', 'Year', 'tic'
compustat2021 = compustat2021.dropna(subset=['cik', 'Year', 'tic'])
print(len(compustat2021))

# Obtain the firm list
file_path = data_root + 'tic_list.csv'
tic_list = pd.read_csv(file_path)
tic_list

C:\Users\Xu Tongzhou\AppData\Local\Temp\ipykernel_12184\3788066661.py:4: DtypeWarning: Columns (10,30,948,975) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat2021 = pd.read_csv(file_path)


12656
8564


,tic
0,WIRE
1,LNT
2,AMR
3,SFM
4,ENSG
...,...
1192,NOV
1193,DIS
1194,PACW
1195,DBD


### Focus on the data for 1197 firm set

In [3]:
# compustat subset that contains the data for 1197 firm set
target_compustat = pd.merge(compustat2021, tic_list, how='right', on='tic')
print(len(target_compustat))

# Some duplicate rows in field tic!!!
target_compustat = target_compustat.drop_duplicates(subset='tic', keep='first')
print(len(target_compustat))
target_compustat

1381
1197


,gvkey,datadate,Year,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,25572,20211231,2021.0,INDL,C,D,STD,WIRE,292562105,ENCORE WIRE CORP,...,1.0,3357,358.0,925.0,B+,TX,0.0,www.encorewire.com,NaN,19920716.0
1,11554,20211231,2021.0,INDL,C,D,STD,LNT,018802108,ALLIANT ENERGY CORP,...,1.0,4931,705.0,700.0,A,WI,0.0,www.alliantenergy.com,NaN,NaN
2,27841,20211231,2021.0,INDL,C,D,STD,AMR,020764106,ALPHA METALLURGICAL RESOURCE,...,1.0,1220,NaN,NaN,NaN,TN,0.0,www.alphametresources.com,NaN,NaN
3,17934,20211231,2021.0,INDL,C,D,STD,SFM,85208M102,SPROUTS FARMERS MARKET,...,1.0,5411,NaN,NaN,NaN,AZ,0.0,www.sprouts.com,NaN,20130801.0
4,178803,20211231,2021.0,INDL,C,D,STD,ENSG,29358P101,ENSIGN GROUP INC,...,1.0,8051,NaN,NaN,A-,CA,0.0,ensigngroup.net,NaN,20071109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,63892,20211231,2021.0,INDL,C,D,STD,NOV,62955J103,NOV INC,...,1.0,3533,395.0,935.0,C,TX,0.0,www.nov.com,NaN,19961028.0
1376,3980,20210930,2021.0,INDL,C,D,STD,DIS,254687106,DISNEY (WALT) CO,...,1.0,4888,245.0,978.0,A,CA,0.0,www.thewaltdisneycompany.com,NaN,NaN
1377,136265,20211231,2021.0,INDL,C,D,STD,PACW,695263103,PACWEST BANCORP,...,1.0,6020,817.0,800.0,B-,CA,0.0,www.pacwestbancorp.com,NaN,20000602.0
1379,3946,20211231,2021.0,INDL,C,D,STD,DBD,253651103,DIEBOLD NIXDORF INC,...,1.0,3578,357.0,925.0,C,OH,0.0,www.dieboldnixdorf.com,NaN,NaN


### Define two functions to filter columns

drop_imbalanced_columns is used to remove columns with excessive data concentration

drop_high_cardinality_columns is used to remove string columns with overly dispersed data

In [4]:
def drop_imbalanced_columns(dataframe, threshold=0.95):
    """
    Remove columns where the sum of the top three values' proportions exceeds the threshold.
    
    Args:
    - dataframe (pd.DataFrame): Input DataFrame.
    - threshold (float): Proportion threshold for the sum of top three values to trigger column deletion (default: 0.95).

    Returns:
    - pd.DataFrame: DataFrame with columns removed.
    """
    cols_to_drop = []
    
    for col in dataframe.columns:
        # Calculate normalized value counts
        proportions = dataframe[col].value_counts(normalize=True, dropna=False)
        
        # Sum the top 3 proportions
        top_3_sum = proportions.nlargest(3).sum()
        
        # Check if the sum exceeds the threshold
        if top_3_sum > threshold:
            cols_to_drop.append(col)
    
    # Drop the identified columns
    return dataframe.drop(columns=cols_to_drop)


def drop_high_cardinality_columns(dataframe, threshold=1100):
    """
    Remove columns of type string with a number of unique values exceeding the threshold.
    
    Args:
    - dataframe (pd.DataFrame): Input DataFrame.
    - threshold (int): Maximum allowed unique values for string columns (default: 1100).
    
    Returns:
    - pd.DataFrame: DataFrame with columns removed.
    """
    cols_to_drop = [
        col for col in dataframe.columns
        if col != 'tic' and dataframe[col].dtype == 'object' and dataframe[col].nunique() > threshold
    ]
    
    # Drop the identified columns
    return dataframe.drop(columns=cols_to_drop)


### Finding the best threshold

In [5]:
# find the best threshold
column_num_list = []
x_axis = range(1, 101)
y_axis = range(0, 901, 20)
for i in x_axis:
    rate = i/100
    column_num_list.append(len(drop_imbalanced_columns(target_compustat, rate).columns))


import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
plt.grid(True)
plt.plot(x_axis, column_num_list)
plt.xticks(ticks=x_axis, labels=x_axis, rotation=45)
plt.yticks(ticks=y_axis, labels=y_axis, rotation=45)
plt.show()

# 0.89, 0.95 seems to be the perfect threshold

KeyboardInterrupt: 

### Execute Filter

In [6]:
target_column = drop_imbalanced_columns(target_compustat, 0.75)
# print(target_column.notna().mean())
len(target_column.columns)

305

In [7]:
target_column = drop_high_cardinality_columns(target_column, 1100)
len(target_column.columns)

297

In [8]:
target_column

,gvkey,tic,fdate,pdate,aco,acodo,acominc,acox,acqgdwl,acqintan,...,gsector,gsubind,incorp,naics,sic,spcindcd,spcseccd,spcsrc,state,ipodate
0,25572,WIRE,20220219.0,20220215.0,3.167,3.167,0.000,3.167,NaN,NaN,...,20.0,20104010.0,DE,331420.0,3357,358.0,925.0,B+,TX,19920716.0
1,11554,LNT,20220221.0,NaN,344.000,NaN,0.000,NaN,NaN,NaN,...,55.0,55101010.0,WI,22111.0,4931,705.0,700.0,A,WI,NaN
2,27841,AMR,20220404.0,20220311.0,32.014,0.000,-58.503,0.462,NaN,NaN,...,15.0,15104050.0,NaN,21211.0,1220,NaN,NaN,NaN,TN,NaN
3,17934,SFM,20220310.0,20220224.0,22.112,0.564,-3.758,0.564,NaN,NaN,...,30.0,30101030.0,DE,445110.0,5411,NaN,NaN,NaN,AZ,20130801.0
4,178803,ENSG,20220214.0,20220211.0,35.014,35.014,0.000,35.014,6.000,0.075,...,35.0,35102020.0,DE,623110.0,8051,NaN,NaN,A-,CA,20071109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,63892,NOV,20220215.0,20220206.0,198.000,198.000,-1546.000,198.000,NaN,NaN,...,10.0,10101020.0,DE,333132.0,3533,395.0,935.0,C,TX,19961028.0
1376,3980,DIS,20211130.0,20211126.0,814.000,814.000,-6440.000,814.000,NaN,NaN,...,50.0,50202010.0,DE,516120.0,4888,245.0,978.0,A,CA,NaN
1377,136265,PACW,20220307.0,NaN,NaN,NaN,65.968,NaN,327.066,34.050,...,40.0,40101015.0,DE,522110.0,6020,817.0,800.0,B-,CA,20000602.0
1379,3946,DBD,20220325.0,20220210.0,324.700,276.500,-378.500,276.500,NaN,NaN,...,45.0,45202030.0,OH,334118.0,3578,357.0,925.0,C,OH,NaN


### Manually delete some columns 

In [37]:
target_column = target_column.drop('au', axis=1)        # Auditor number
target_column = target_column.drop('addzip', axis=1)    # Postal Code
target_column = target_column.drop('city', axis=1)
target_column = target_column.drop('ggroup', axis=1)    # GIC Groups
target_column = target_column.drop('gind', axis=1)      # GIC Industries
target_column = target_column.drop('gsector', axis=1)   # GIC Sector
target_column = target_column.drop('gsubind', axis=1)   # GIC Sub-Industries
target_column = target_column.drop('incorp', axis=1)    # Current State/Province of Incorporation Code
target_column = target_column.drop('naics', axis=1)     # North American Industry Classication Code
target_column = target_column.drop('sic', axis=1)       # Standard Industry Classication Code
target_column = target_column.drop('spcindcd', axis=1)  # S&P Industry Sector Code
target_column = target_column.drop('spcseccd', axis=1)  # S&P Economic Sector Code
target_column = target_column.drop('state', axis=1)     # State/Province
target_column = target_column.drop('ipodate', axis=1)   # Company Initial Public Oering Date
target_column = target_column.drop('gvkey', axis=1)     # Global Company Key
target_column = target_column.drop('fdate', axis=1)     # Final Date
target_column = target_column.drop('pdate', axis=1)     # Preliminary Date


### spcsrc adjustment

spcsrc: S&P Quality Ranking

mapping: {
    "A+": 1,
    "A": 2,
    "A-": 3,
    "B+": 4,
    "B": 5,
    "B-": 6,
    "C": 7,
    "D": 8
}

In [25]:
# Only 2 types of columns left
print(target_column.dtypes.unique())

# Only tic and spcsrc are the object type column
print(target_column.select_dtypes(include=['object']))

# Unique values in spcsrc
print(target_column['spcsrc'].value_counts(normalize=True, dropna=False))

[dtype('O') dtype('float64')]
       tic spcsrc
0     WIRE     B+
1      LNT      A
2      AMR    NaN
3      SFM    NaN
4     ENSG     A-
...    ...    ...
1375   NOV      C
1376   DIS      A
1377  PACW     B-
1379   DBD      C
1380   UAL     B-

[1197 rows x 2 columns]
spcsrc
B+     0.219716
B      0.177945
C      0.150376
B-     0.144528
NaN    0.111947
A-     0.094403
A      0.066834
A+     0.022556
D      0.011696
Name: proportion, dtype: float64


In [28]:
spcsrc_mapping = {
    "A+": 1,
    "A": 2,
    "A-": 3,
    "B+": 4,
    "B": 5,
    "B-": 6,
    "C": 7,
    "D": 8
}

target_column['spcsrc_num'] = target_column['spcsrc'].map(spcsrc_mapping)
target_column = target_column.drop('spcsrc', axis=1)

In [36]:
# Little test
for col in target_column.columns:
    if len(target_column[col].value_counts(normalize=True, dropna=False)) < 40:
        print(target_column[col].value_counts(normalize=True, dropna=False), '\n\n\n')

au
4.0     0.253133
5.0     0.234754
7.0     0.221387
6.0     0.157059
17.0    0.055973
11.0    0.020886
21.0    0.015873
9.0     0.012531
16.0    0.011696
12.0    0.006683
23.0    0.004177
25.0    0.001671
NaN     0.001671
18.0    0.000835
26.0    0.000835
24.0    0.000835
Name: proportion, dtype: float64 



ggroup
2010.0    0.142857
1510.0    0.087719
3510.0    0.055973
1010.0    0.055138
2550.0    0.052632
4010.0    0.052632
2530.0    0.045948
2520.0    0.045113
2020.0    0.040936
4020.0    0.040936
4510.0    0.040936
6010.0    0.038429
5510.0    0.037594
4030.0    0.036759
4520.0    0.035088
3520.0    0.035088
2030.0    0.033417
5020.0    0.030075
3020.0    0.029240
2510.0    0.020886
4530.0    0.017544
3010.0    0.010860
3030.0    0.008354
5010.0    0.005848
Name: proportion, dtype: float64 



gsector
20.0    0.217210
25.0    0.164578
40.0    0.130326
45.0    0.093567
35.0    0.091061
15.0    0.087719
10.0    0.055138
30.0    0.048454
60.0    0.038429
55.0    0.037594
50.0    0.

In [30]:
# Save
target_column.to_csv("target_column.csv", index=False)

## About the duplicates in tic field

In [33]:
target_compustat = pd.merge(compustat2021, tic_list, how='right', on='tic')
target_compustat

,gvkey,datadate,Year,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,25572,20211231,2021.0,INDL,C,D,STD,WIRE,292562105,ENCORE WIRE CORP,...,1.0,3357,358.0,925.0,B+,TX,0.0,www.encorewire.com,NaN,19920716.0
1,11554,20211231,2021.0,INDL,C,D,STD,LNT,018802108,ALLIANT ENERGY CORP,...,1.0,4931,705.0,700.0,A,WI,0.0,www.alliantenergy.com,NaN,NaN
2,27841,20211231,2021.0,INDL,C,D,STD,AMR,020764106,ALPHA METALLURGICAL RESOURCE,...,1.0,1220,NaN,NaN,NaN,TN,0.0,www.alphametresources.com,NaN,NaN
3,17934,20211231,2021.0,INDL,C,D,STD,SFM,85208M102,SPROUTS FARMERS MARKET,...,1.0,5411,NaN,NaN,NaN,AZ,0.0,www.sprouts.com,NaN,20130801.0
4,178803,20211231,2021.0,INDL,C,D,STD,ENSG,29358P101,ENSIGN GROUP INC,...,1.0,8051,NaN,NaN,A-,CA,0.0,ensigngroup.net,NaN,20071109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,3980,20210930,2021.0,INDL,C,D,STD,DIS,254687106,DISNEY (WALT) CO,...,1.0,4888,245.0,978.0,A,CA,0.0,www.thewaltdisneycompany.com,NaN,NaN
1377,136265,20211231,2021.0,INDL,C,D,STD,PACW,695263103,PACWEST BANCORP,...,1.0,6020,817.0,800.0,B-,CA,0.0,www.pacwestbancorp.com,NaN,20000602.0
1378,136265,20211231,2021.0,FS,C,D,STD,PACW,695263103,PACWEST BANCORP,...,1.0,6020,817.0,800.0,B-,CA,0.0,www.pacwestbancorp.com,NaN,20000602.0
1379,3946,20211231,2021.0,INDL,C,D,STD,DBD,253651103,DIEBOLD NIXDORF INC,...,1.0,3578,357.0,925.0,C,OH,0.0,www.dieboldnixdorf.com,NaN,NaN


In [34]:

duplicates = target_compustat[target_compustat.duplicated('tic', keep=False)]
duplicates

,gvkey,datadate,Year,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
19,144009,20211231,2021.0,FS,C,D,STD,WTW,G96629103,WILLIS TOWERS WATSON PLC,...,1.0,6411,330.0,800.0,B+,NaN,0.0,www.willistowerswatson.com,NaN,NaN
20,144009,20211231,2021.0,INDL,C,D,STD,WTW,G96629103,WILLIS TOWERS WATSON PLC,...,1.0,6411,330.0,800.0,B+,NaN,0.0,www.willistowerswatson.com,NaN,NaN
36,174729,20211231,2021.0,INDL,C,D,STD,TNL,894164102,TRAVEL PLUS LEISURE CO,...,1.0,6531,NaN,NaN,B+,FL,0.0,www.travelandleisureco.com,NaN,20060801.0
37,174729,20211231,2021.0,FS,C,D,STD,TNL,894164102,TRAVEL PLUS LEISURE CO,...,1.0,6531,NaN,NaN,B+,FL,0.0,www.travelandleisureco.com,NaN,20060801.0
43,4674,20211231,2021.0,INDL,C,D,STD,RF,7591EP100,REGIONS FINANCIAL CORP,...,1.0,6020,815.0,800.0,A-,AL,0.0,www.regions.com,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,26016,20211231,2021.0,FS,C,D,STD,UVE,91359V107,UNIVERSAL INSURANCE HLDGS,...,1.0,6331,835.0,800.0,B+,FL,0.0,www.universalinsuranceholdings.com,NaN,19921214.0
1368,14822,20211231,2021.0,INDL,C,D,STD,WRB,084423102,BERKLEY (W R) CORP,...,1.0,6331,835.0,800.0,B+,CT,0.0,www.berkley.com,NaN,NaN
1369,14822,20211231,2021.0,FS,C,D,STD,WRB,084423102,BERKLEY (W R) CORP,...,1.0,6331,835.0,800.0,B+,CT,0.0,www.berkley.com,NaN,NaN
1377,136265,20211231,2021.0,INDL,C,D,STD,PACW,695263103,PACWEST BANCORP,...,1.0,6020,817.0,800.0,B-,CA,0.0,www.pacwestbancorp.com,NaN,20000602.0


In [36]:
grouped = duplicates.groupby('tic')

# Check if all values in 'Age' and 'City' are the same for each group
consistency = grouped.agg(lambda x: len(x.unique()) == 1)
print(consistency)

      gvkey  datadate  Year  indfmt  consol  popsrc  datafmt  cusip  conm  \
tic                                                                         
AAIC   True      True  True   False    True    True     True   True  True   
ACC    True      True  True   False    True    True     True   True  True   
AFG    True      True  True   False    True    True     True   True  True   
AFL    True      True  True   False    True    True     True   True  True   
AIG    True      True  True   False    True    True     True   True  True   
...     ...       ...   ...     ...     ...     ...      ...    ...   ...   
WSFS   True      True  True   False    True    True     True   True  True   
WTFC   True      True  True   False    True    True     True   True  True   
WTW    True      True  True   False    True    True     True   True  True   
WU     True      True  True   False    True    True     True   True  True   
ZION   True      True  True   False    True    True     True   True  True   

In [41]:
consistent_num = consistency.sum()
print(consistent_num[consistent_num != 184])
print(consistent_num)

indfmt       0
acctchg    183
acctstd     77
acqmeth    171
bspr       161
          ... 
xuwti      145
auop       181
auopic      62
ceoso       62
cfoso       62
Length: 737, dtype: int64
gvkey       184
datadate    184
Year        184
indfmt        0
consol      184
           ... 
state       184
stko        184
weburl      184
dldte       184
ipodate     184
Length: 980, dtype: int64
